In [0]:
#--------------------------------------------------------------------------------------------------------------
!pip install open3d
!pip install nuscenes-devkit
#--------------------------------------------------------------------------------------------------------------

In [0]:
#--------------------------------------------------------------------------------------------------------------
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud, Box
import pandas as pd
import numpy as np
import glob
import cv2
import os
import open3d as o3d
#--------------------------------------------------------------------------------------------------------------

In [2]:
#--------------------------------------------------------------------------------------------------------------
datafile = "./drive/My Drive/v1.0-mini/data/Final_Data_2_Changed.csv"
#datafile = "./drive/My Drive/v1.0-mini/data/Final_Sample_Wise_changed.csv"
dataframe = pd.read_csv(datafile)
dataframe.head()
#--------------------------------------------------------------------------------------------------------------

,index,lidar_top,radar_front,radar_front_left,radar_front_right,radar_back_left,radar_back_right,montage_image,bbox_size,num_lidar_pts,num_radar_pts,visibility_token,category_name
0,0,samples/LIDAR_TOP/n015-2018-07-24-11-22-45+080...,samples/RADAR_FRONT/n015-2018-07-24-11-22-45+0...,samples/RADAR_FRONT_LEFT/n015-2018-07-24-11-22...,samples/RADAR_FRONT_RIGHT/n015-2018-07-24-11-2...,samples/RADAR_BACK_LEFT/n015-2018-07-24-11-22-...,samples/RADAR_BACK_RIGHT/n015-2018-07-24-11-22...,./drive/My Drive/v1.0-mini/samples/MONTAGE/sce...,"[0.621, 0.669, 1.642]",1,0,1,pedestrian
1,1,samples/LIDAR_TOP/n015-2018-07-24-11-22-45+080...,samples/RADAR_FRONT/n015-2018-07-24-11-22-45+0...,samples/RADAR_FRONT_LEFT/n015-2018-07-24-11-22...,samples/RADAR_FRONT_RIGHT/n015-2018-07-24-11-2...,samples/RADAR_BACK_LEFT/n015-2018-07-24-11-22-...,samples/RADAR_BACK_RIGHT/n015-2018-07-24-11-22...,./drive/My Drive/v1.0-mini/samples/MONTAGE/sce...,"[0.775, 0.769, 1.711]",2,0,2,pedestrian
2,2,samples/LIDAR_TOP/n015-2018-07-24-11-22-45+080...,samples/RADAR_FRONT/n015-2018-07-24-11-22-45+0...,samples/RADAR_FRONT_LEFT/n015-2018-07-24-11-22...,samples/RADAR_FRONT_RIGHT/n015-2018-07-24-11-2...,samples/RADAR_BACK_LEFT/n015-2018-07-24-11-22-...,samples/RADAR_BACK_RIGHT/n015-2018-07-24-11-22...,./drive/My Drive/v1.0-mini/samples/MONTAGE/sce...,"[2.011, 4.633, 1.573]",5,0,3,vehicle_small
3,3,samples/LIDAR_TOP/n015-2018-07-24-11-22-45+080...,samples/RADAR_FRONT/n015-2018-07-24-11-22-45+0...,samples/RADAR_FRONT_LEFT/n015-2018-07-24-11-22...,samples/RADAR_FRONT_RIGHT/n015-2018-07-24-11-2...,samples/RADAR_BACK_LEFT/n015-2018-07-24-11-22-...,samples/RADAR_BACK_RIGHT/n015-2018-07-24-11-22...,./drive/My Drive/v1.0-mini/samples/MONTAGE/sce...,"[0.752, 0.819, 1.637]",1,0,4,pedestrian
4,4,samples/LIDAR_TOP/n015-2018-07-24-11-22-45+080...,samples/RADAR_FRONT/n015-2018-07-24-11-22-45+0...,samples/RADAR_FRONT_LEFT/n015-2018-07-24-11-22...,samples/RADAR_FRONT_RIGHT/n015-2018-07-24-11-2...,samples/RADAR_BACK_LEFT/n015-2018-07-24-11-22-...,samples/RADAR_BACK_RIGHT/n015-2018-07-24-11-22...,./drive/My Drive/v1.0-mini/samples/MONTAGE/sce...,"[0.427, 0.359, 0.794]",1,0,4,small_objects


In [0]:
#--------------------------------------------------------------------------------------------------------------
def load_imagesAll(dataframe):
  images = []
  for file in dataframe:
    image = cv2.imread(file)
    image = cv2.resize(image,(128,128))
    images.append(image)
  
  return np.array(images)
#--------------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------------
def readLidarData(dataframe):
  data = []
  path = "./drive/My Drive/v1.0-mini/"
  for file in dataframe:
    pt = LidarPointCloud.from_file(path+file)
    data.append([np.sum(pt.points[0]), np.sum(pt.points[1]), np.sum(pt.points[2]), np.sum(pt.points[3])])
  #print("Lidar Data Finished Loading..")
  return np.array(data)
#--------------------------------------------------------------------------------------------------------------
def readRadarData(dataframe):
  data = []
  path = "./drive/My Drive/v1.0-mini/"
  for file in dataframe:
    pcd = o3d.io.read_point_cloud(path+file)
    downpcd = pcd.voxel_down_sample(voxel_size=0.05)
    temp = downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=4))
    d1 = np.asarray(np.sum(downpcd.normals))

    pt = RadarPointCloud.from_file(path+file)

    data.append([np.sum(pt.points), d1])
  #print("Radar Data Finished Loading..")
  return np.array(data)
#--------------------------------------------------------------------------------------------------------------
def readAllPointClout(dataframe):
  data = readLidarData(dataframe['lidar_top'])
  #print("Lidar Data Shape:", data.shape)

  rdata1 = readRadarData(dataframe['radar_front'])
  #print("Radar Data 1 Shape:", rdata1.shape)
  rdata2 = readRadarData(dataframe['radar_front_left'])
  #print("Radar Data 2 Shape:", rdata2.shape)
  rdata3 = readRadarData(dataframe['radar_front_right'])
  #print("Radar Data 3 Shape:", rdata3.shape)
  rdata4 = readRadarData(dataframe['radar_back_left'])
  #print("Radar Data 4 Shape:", rdata4.shape)
  rdata5 = readRadarData(dataframe['radar_back_right'])
  #print("Radar Data 5 Shape:", rdata5.shape)

  finaldata = np.column_stack((data, rdata1, rdata2, rdata3, rdata4, rdata5))
  #print("Final Data Shape", finaldata.shape)
  return np.array(finaldata)
#--------------------------------------------------------------------------------------------------------------

In [11]:
#------------------------------------------------------------------
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
#------------------------------------------------------------------
# Data Preprocessing
#------------------------------------------------------------------
# Encoding Labels
labels = dataframe['category_name']
le = LabelEncoder()
le.fit(labels)
labels_encoded = le.transform(labels)
le.classes_
len(labels_encoded)
#------------------------------------------------------------------

18538

In [0]:
#------------------------------------------------------------------
# Creating train/test split
from sklearn.model_selection import train_test_split
#------------------------------------------------------------------
X_train, X_test1, y_train, y_test1 = train_test_split(dataframe, labels_encoded, test_size=0.10, random_state=1992)
#------------------------------------------------------------------
X_train, X_test2, y_train, y_test2 = train_test_split(dataframe, labels_encoded, test_size=0.20, random_state=1992)
#------------------------------------------------------------------
X_train, X_test3, y_train, y_test3 = train_test_split(dataframe, labels_encoded, test_size=0.30, random_state=1992)
#------------------------------------------------------------------
X_train, X_test4, y_train, y_test4 = train_test_split(dataframe, labels_encoded, test_size=0.40, random_state=1992)
#------------------------------------------------------------------

In [6]:
#--------------------------------------------------------------------------------------------------------------
# Loading Data
#--------------------------------------------------------------------------------------------------------------
print("Loading Image Data..")
images_test1  = load_imagesAll(X_test1 ['montage_image'])
images_test2  = load_imagesAll(X_test2 ['montage_image'])
images_test3  = load_imagesAll(X_test3 ['montage_image'])
images_test4  = load_imagesAll(X_test4 ['montage_image'])
#--------------------------------------------------------------------------------------------------------------
print("Continious Data..")
cont_data_test1  = X_test1 [['num_lidar_pts','num_radar_pts','visibility_token']]
cont_data_test2  = X_test2 [['num_lidar_pts','num_radar_pts','visibility_token']]
cont_data_test3  = X_test3 [['num_lidar_pts','num_radar_pts','visibility_token']]
cont_data_test4  = X_test4 [['num_lidar_pts','num_radar_pts','visibility_token']]
#--------------------------------------------------------------------------------------------------------------
print("Loading Pointcloud Data..")
pc_test1  = readAllPointClout(X_test1)
pc_test2  = readAllPointClout(X_test2)
pc_test3  = readAllPointClout(X_test3)
pc_test4  = readAllPointClout(X_test4)
#--------------------------------------------------------------------------------------------------------------
print("Successfully finised loading all data..")

Loading Image Data..
Continious Data..
Loading Pointcloud Data..
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WAR

In [0]:
mnScaler = sklearn.preprocessing.MinMaxScaler()
#--------------------------------------------------------------------------------------------------------------
# Normalizing continious data
cont_data_test1  = pd.DataFrame(mnScaler.fit_transform(cont_data_test1))
cont_data_test2  = pd.DataFrame(mnScaler.fit_transform(cont_data_test2))
cont_data_test3  = pd.DataFrame(mnScaler.fit_transform(cont_data_test3))
cont_data_test4  = pd.DataFrame(mnScaler.fit_transform(cont_data_test4))

#--------------------------------------------------------------------------------------------------------------
# Normalizing pointcloud data
#--------------------------------------------------------------------------------------------------------------
pc_test_norm1  = pd.DataFrame(mnScaler.fit_transform(pc_test1))
pc_test_norm2  = pd.DataFrame(mnScaler.fit_transform(pc_test2))
pc_test_norm3  = pd.DataFrame(mnScaler.fit_transform(pc_test3))
pc_test_norm4  = pd.DataFrame(mnScaler.fit_transform(pc_test4))
#--------------------------------------------------------------------------------------------------------------

In [0]:
#------------------------------------------------------------
# importing libraries
#------------------------------------------------------------
import keras
from keras.optimizers import Adam
import matplotlib.pyplot as plt

from keras import layers, preprocessing
from keras.models import Model
from keras.layers import *
from keras.layers import Dense, Dropout, Input
#------------------------------------------------------------

In [0]:
#------------------------------------------------------------------
# Loading Previously Created Models
ContModel = keras.models.load_model("./drive/My Drive/v1.0-mini/model/cont_model.h5")
ImgModel  = keras.models.load_model("./drive/My Drive/v1.0-mini/model/image_model.h5")
PCModel   = keras.models.load_model("./drive/My Drive/v1.0-mini/model/pc_model.h5")
CombModel = keras.models.load_model("./drive/My Drive/v1.0-mini/model/combined_model.h5")
#------------------------------------------------------------------

In [15]:
batch_size = 64
#------------------------------------------------------------------
# Model Testing and evaluation
#------------------------------------------------------------------
print("Continious Model Prediction..")
lossCN, evalCN = ContModel.evaluate(cont_data_test1, y_test1, batch_size=batch_size)
print("Image Model Prediction..")
lossIM, evalIM = ImgModel.evaluate(images_test1, y_test1, batch_size=batch_size)
print("Pointcloud Model Prediction..")
lossPC, evalPC = PCModel.evaluate(pc_test_norm1, y_test1, batch_size=batch_size)
print("Combined Model Prediction..")
lossCB, evalCB = CombModel.evaluate([cont_data_test1, images_test1, pc_test_norm1], y_test1,
                                         batch_size=batch_size)

Continious Model Prediction..
1854/1854 [==============================] - 0s 127us/step
Image Model Prediction..
1854/1854 [==============================] - 2s 953us/step
Pointcloud Model Prediction..
1854/1854 [==============================] - 0s 140us/step
Combined Model Prediction..
1854/1854 [==============================] - 2s 1ms/step


In [0]:
#------------------------------------------------------------------
finalpred = (ImagePrediction*0.4 + PCPrediction*0.35 + ContPrediction*0.25)
#print("Final weighted Average", finalpred)
np.max(finalpred, axis=0)

In [19]:
print("loss:", lossCB, "\t accuracy:", evalCB)
print("loss:", lossCN, "\t accuracy:", evalCN)
print("loss:", lossIM, "\t accuracy:", evalIM)
print("loss:", lossPC, "\t accuracy:", evalPC)

loss: 0.9177630896707183 	 accuracy: 0.6003236174583435
loss: 1.0902681872811262 	 accuracy: 0.517259955406189
loss: 0.9273311189720417 	 accuracy: 0.608953595161438
loss: 0.9933007592265017 	 accuracy: 0.5609493255615234


In [20]:
batch_size = 64
#------------------------------------------------------------------
# Model Testing and evaluation
#------------------------------------------------------------------
print("Continious Model Prediction..")
lossCN, evalCN = ContModel.evaluate(cont_data_test2, y_test2, batch_size=batch_size)
print("Image Model Prediction..")
lossIM, evalIM = ImgModel.evaluate(images_test2, y_test2, batch_size=batch_size)
print("Pointcloud Model Prediction..")
lossPC, evalPC = PCModel.evaluate(pc_test_norm2, y_test2, batch_size=batch_size)
print("Combined Model Prediction..")
lossCB, evalCB = CombModel.evaluate([cont_data_test2, images_test2, pc_test_norm2], y_test2,
                                         batch_size=batch_size)


print("loss:", lossCB, "\t accuracy:", evalCB)
print("loss:", lossCN, "\t accuracy:", evalCN)
print("loss:", lossIM, "\t accuracy:", evalIM)
print("loss:", lossPC, "\t accuracy:", evalPC)

Continious Model Prediction..
3708/3708 [==============================] - 0s 50us/step
Image Model Prediction..
3708/3708 [==============================] - 3s 858us/step
Pointcloud Model Prediction..
3708/3708 [==============================] - 0s 53us/step
Combined Model Prediction..
3708/3708 [==============================] - 3s 826us/step
loss: 0.9063292171448437 	 accuracy: 0.6084142327308655
loss: 1.094100970151998 	 accuracy: 0.5188781023025513
loss: 0.9208443834172098 	 accuracy: 0.6094930171966553
loss: 0.9823778665181503 	 accuracy: 0.5693095922470093


In [21]:
batch_size = 64
#------------------------------------------------------------------
# Model Testing and evaluation
#------------------------------------------------------------------
print("Continious Model Prediction..")
lossCN, evalCN = ContModel.evaluate(cont_data_test3, y_test3, batch_size=batch_size)
print("Image Model Prediction..")
lossIM, evalIM = ImgModel.evaluate(images_test3, y_test3, batch_size=batch_size)
print("Pointcloud Model Prediction..")
lossPC, evalPC = PCModel.evaluate(pc_test_norm3, y_test3, batch_size=batch_size)
print("Combined Model Prediction..")
lossCB, evalCB = CombModel.evaluate([cont_data_test3, images_test3, pc_test_norm3], y_test3,
                                         batch_size=batch_size)


print("loss:", lossCB, "\t accuracy:", evalCB)
print("loss:", lossCN, "\t accuracy:", evalCN)
print("loss:", lossIM, "\t accuracy:", evalIM)
print("loss:", lossPC, "\t accuracy:", evalPC)

Continious Model Prediction..
5562/5562 [==============================] - 0s 52us/step
Image Model Prediction..
5562/5562 [==============================] - 5s 826us/step
Pointcloud Model Prediction..
5562/5562 [==============================] - 0s 55us/step
Combined Model Prediction..
5562/5562 [==============================] - 5s 822us/step
loss: 0.9175457082734353 	 accuracy: 0.6066163182258606
loss: 1.1020548076605634 	 accuracy: 0.5152822732925415
loss: 0.9334892677943598 	 accuracy: 0.6001438498497009
loss: 0.9953694615120598 	 accuracy: 0.5641855597496033


In [22]:
batch_size = 64
#------------------------------------------------------------------
# Model Testing and evaluation
#------------------------------------------------------------------
print("Continious Model Prediction..")
lossCN, evalCN = ContModel.evaluate(cont_data_test4, y_test4, batch_size=batch_size)
print("Image Model Prediction..")
lossIM, evalIM = ImgModel.evaluate(images_test4, y_test4, batch_size=batch_size)
print("Pointcloud Model Prediction..")
lossPC, evalPC = PCModel.evaluate(pc_test_norm4, y_test4, batch_size=batch_size)
print("Combined Model Prediction..")
lossCB, evalCB = CombModel.evaluate([cont_data_test4, images_test4, pc_test_norm4], y_test4,
                                         batch_size=batch_size)


print("loss:", lossCB, "\t accuracy:", evalCB)
print("loss:", lossCN, "\t accuracy:", evalCN)
print("loss:", lossIM, "\t accuracy:", evalIM)
print("loss:", lossPC, "\t accuracy:", evalPC)

Continious Model Prediction..
7416/7416 [==============================] - 0s 49us/step
Image Model Prediction..
7416/7416 [==============================] - 6s 818us/step
Pointcloud Model Prediction..
7416/7416 [==============================] - 0s 56us/step
Combined Model Prediction..
7416/7416 [==============================] - 6s 833us/step
loss: 0.9251100422237812 	 accuracy: 0.6035598516464233
loss: 1.105617069632233 	 accuracy: 0.5146979689598083
loss: 0.9429650642339465 	 accuracy: 0.5931769013404846
loss: 1.0065209421910901 	 accuracy: 0.5629719495773315
